In [68]:
import os
import pandas as pd
import numpy as np 
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret

# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [69]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-06.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  # Gets 'regional-au-daily-2024-02-05.csv'
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [70]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [71]:
# Assuming `track_ids` is your Pandas Series of track IDs as shown
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialize lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that you have all the data, you can put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame to verify the results
popularity_df.head()


,artist_name,track_name,popularity_score
0,Noah Kahan,Stick Season,97
1,Benson Boone,Beautiful Things,94
2,Jack Harlow,Lovin On Me,99
3,Tate McRae,greedy,100
4,Taylor Swift,Cruel Summer,99


In [72]:
df['popularity_score'] = popularity_df['popularity_score']

In [73]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [74]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,06-02-2024,1,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,124,346140,97
1,06-02-2024,2,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,3,19,299738,94
2,06-02-2024,3,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,2,89,285482,99
3,06-02-2024,4,Tate McRae,greedy,3rUGC1vUpkDG9CZFHMur1t,RCA Records Label,1,4,145,239056,100
4,06-02-2024,5,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,5,367,230698,99


In [75]:
df['previous_rank'].unique()

array([  1,   3,   2,   4,   5,   7,   6,   8,   9,  10,  11,  12,  14,
        13,  15,  16,  18,  17,  19,  20,  21,  22,  23,  24,  32,  35,
        25,  30,  28,  26,  29,  27,  34,  31,  36,  39,  38,  37,  42,
        33,  40,  44,  47,  49,  41,  46,  45,  43,  48,  50,  52,  53,
        51,  58,  56,  55,  54,  57,  59,  63,  60,  61,  62,  66,  68,
        64,  70,  67, 103,  69,  65,  83,  71,  78,  72,  80,  73,  74,
        84,  75,  82,  90,  76,  85,  79,  96,  86,  91,  88,  77,  89,
        92,  81,  99,  87,  94, 106,  95,  -1, 114,  97,  93, 102, 101,
       108,  98, 100, 110, 104, 130, 120, 118, 109, 151, 105, 112, 145,
       107, 115, 113, 111, 123, 139, 122, 119, 153, 124, 143, 141, 129,
       117, 116, 121, 131, 133, 132, 134, 135, 126, 136, 127, 125, 137,
       144, 149, 140, 157, 128, 152, 156, 158, 138, 162, 160, 159, 142,
       155, 165, 170, 182, 168, 147, 167, 173, 154, 175, 146, 161, 150,
       148, 163, 177, 183, 181, 174, 166, 178, 186, 184, 172, 16

Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [76]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [77]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

In [78]:
# Selecting specific columns to view
selected_columns_df = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]

# Display the DataFrame with the selected columns
selected_columns_df['rank_change'].unique()


array(['=', '+1', '-1', '+7', '+9', '-2', '+2', '-4', '-5', '-3', '+3',
       '-7', '+4', '+5', '+34', '-6', '+11', '+8', '+10', '-13', '-12',
       '-8', 'RE', '+14', '-9', '+20', '+6', '+37', '-10', '+28', '-11',
       '+16', '+27', '+15', '+12', '-14', '-16', '-17', '-20', '+21',
       '-22', '-23', 'NEW'], dtype=object)

Need to change "=" to "0", remove "+", and handle "RE"

In [79]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "RE" at position 98
Non-numeric values found:
      rank rank_change                                       artist_title  \
98     99          RE                       Sabrina Carpenter - Nonsense   
159   160          RE                                 Lil Tecca - 500lbs   
174   175          RE   Taylor Swift - You’re Losing Me (From The Vault)   
179   180          RE                    Tate McRae - you broke me first   
184   185          RE  The Beatles - Here Comes The Sun - Remastered ...   
194   195          RE  Elton John - Rocket Man (I Think It's Going To...   
197   198          RE       Daryl Hall & John Oates - You Make My Dreams   
199   200         NEW                              Rich Amiri - ONE CALL   

     days_on_chart  peak_rank days_at_peak  daily_streams  \
98             274         10          NaN          62499   
159             26        134          NaN          50852   
174             5

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_2467/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [80]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [81]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Noah Kahan - Stick Season,0,11.0,12752.0,2364357,-12473,21357413
1,Benson Boone - Beautiful Things,1,1.0,31966.0,1718454,107069,3305539
2,Jack Harlow - Lovin On Me,-1,72.0,12600.0,2033209,-37031,28503421


In [82]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [83]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [84]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [85]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,06-02-2024,Noah Kahan,Stick Season,1,1,0,1,11.0,124,97,346140,12752.0,2364357,-12473,21357413,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,06-02-2024,Benson Boone,Beautiful Things,2,3,1,2,1.0,19,94,299738,31966.0,1718454,107069,3305539,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
2,06-02-2024,Jack Harlow,Lovin On Me,3,2,-1,1,72.0,89,99,285482,12600.0,2033209,-37031,28503421,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
3,06-02-2024,Tate McRae,greedy,4,4,0,1,9.0,145,100,239056,6753.0,1648205,-6044,32968324,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t
4,06-02-2024,Taylor Swift,Cruel Summer,5,5,0,1,1.0,367,99,230698,9832.0,1573496,13092,51656011,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          44 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  192 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [90]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,05-02-2024,Noah Kahan,Stick Season,1,1,0,1,10.0,123,97,333388,8284.0,2376830,-13316,21011273,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,05-02-2024,Jack Harlow,Lovin On Me,2,2,0,1,72.0,88,99,272882,1522.0,2070240,-45409,28217939,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
2,05-02-2024,Benson Boone,Beautiful Things,3,3,0,3,3.0,18,93,267772,28872.0,1611385,91911,3005801,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
3,05-02-2024,Tate McRae,greedy,4,4,0,1,9.0,144,100,232303,1031.0,1654249,-4706,32729268,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t
4,05-02-2024,Taylor Swift,Cruel Summer,5,5,0,1,1.0,366,99,220866,-2740.0,1560404,1221,51425313,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr


In [91]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [94]:
print(len(combined_df))

400


In [67]:
combined_df.to_csv('combined_data/combined.csv', index=False)